<img src="poet.jpeg" alt="Drawing" style="width: 400px;"/> 
<h2><center>Arabic Poetry Dataset

# Import Libraries

In [ ]:
#Python libraries
#Classic,data manipulation 
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score,accuracy_score

# Plots
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import markovify
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import strip_harakat
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from ar_wordcloud import ArabicWordCloud

import pyarabic.arabrepr
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
from tashaphyne.stemming import ArabicLightStemmer
import seaborn as sns
import pyarabic.araby as araby
import pyarabic.number as number
from nltk.corpus import stopwords
import string 

from tqdm import tqdm
import pyarabic.araby as araby

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)

# Read DataSet

In [ ]:
df= pd.read_csv('Arabic Poem Comprehensive Dataset (APCD).csv')

In [ ]:
df=df.iloc[:100000,:]

In [ ]:
df.head()

# EDA 1-1: Features viualizing

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
Most_Generation = df['العصر'].value_counts().sort_values(ascending=False ).to_frame().reset_index()
Most_Generation

In [ ]:
zlbl= get_display( arabic_reshaper.reshape('اكثر الاشعار كانت في عصر'))
plt.figure(figsize=[8,4])
plt.title(zlbl)
df_tuple4 =list(zip(Most_Generation['index'],Most_Generation['العصر']))

first_columns= [get_display( arabic_reshaper.reshape(x[0])) for x in df_tuple4]
second_columns= [x[1] for x in df_tuple4]

xlbl = get_display( arabic_reshaper.reshape('index'))
ylbl = get_display( arabic_reshaper.reshape('العصر'))


sns.barplot(x = second_columns ,y=first_columns,data=Most_Generation,color = "DarkRed");
plt.xlabel(xlbl, fontdict=None, labelpad=None)
plt.ylabel(ylbl, fontdict=None, labelpad=None);

In [ ]:
count_poet = df['الشاعر'].value_counts().sort_values(ascending=False ).to_frame().reset_index()
count_poet

In [ ]:
import seaborn as sns
from bidi.algorithm import get_display

In [ ]:
import arabic_reshaper

In [ ]:
zlbl= get_display( arabic_reshaper.reshape('أكثر القصائد للشعراء'))
plt.figure(figsize=[11,11])
plt.title(zlbl)
df_tuple =list(zip(count_poet['index'],count_poet['الشاعر']))

first_columns= [get_display( arabic_reshaper.reshape(x[0])) for x in df_tuple]
second_columns= [x[1] for x in df_tuple]

xlbl = get_display( arabic_reshaper.reshape('index'))
ylbl = get_display( arabic_reshaper.reshape('الشاعر'))


sns.barplot(x = second_columns ,y=first_columns,data=count_poet,color = "DarkRed");
plt.xlabel(xlbl, fontdict=None, labelpad=None)
plt.ylabel(ylbl, fontdict=None, labelpad=None);

In [ ]:
df.الديوان.value_counts()

In [ ]:
count_rhyme = df['القافية'].value_counts().sort_values(ascending=False ).to_frame().reset_index()
count_rhyme

In [ ]:
zlbl= get_display( arabic_reshaper.reshape('ًاكثر القرافي تكرارا'))
plt.figure(figsize=[11,11])
plt.title(zlbl)
df_tuple2 =list(zip(count_rhyme['index'],count_rhyme['القافية']))

first_columns= [get_display( arabic_reshaper.reshape(x[0])) for x in df_tuple2]
second_columns= [x[1] for x in df_tuple2]

xlbl = get_display( arabic_reshaper.reshape('index'))
ylbl = get_display( arabic_reshaper.reshape('القافية'))


sns.barplot(x = second_columns ,y=first_columns,data=count_rhyme,color = "DarkRed");
plt.xlabel(xlbl, fontdict=None, labelpad=None)
plt.ylabel(ylbl, fontdict=None, labelpad=None);

In [ ]:
count_poem_type = df['البحر'].value_counts().sort_values(ascending=False ).to_frame().reset_index()
count_poem_type

In [ ]:
zlbl= get_display( arabic_reshaper.reshape('اكثر بحور الشعر استخدماً'))
plt.figure(figsize=[11,9])
plt.title(zlbl)
df_tuple3 =list(zip(count_poem_type['index'],count_poem_type['البحر']))

first_columns= [get_display( arabic_reshaper.reshape(x[0])) for x in df_tuple3]
second_columns= [x[1] for x in df_tuple3]

xlbl = get_display( arabic_reshaper.reshape('index'))
ylbl = get_display( arabic_reshaper.reshape('البحر'))


sns.barplot(x = second_columns ,y=first_columns,data=count_poem_type ,color = "DarkRed");
plt.xlabel(xlbl, fontdict=None, labelpad=None)
plt.ylabel(ylbl, fontdict=None, labelpad=None);

# EDA 1-2: Data cleaning

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(how='any',inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.info()

In [ ]:
df.columns

# EDA1-3: Text Preprocessing

> Removing stop words

> Tokenization: Words, ngrams

> Stemming or Lemmatizations 

> Part of speech

> Word corrections

> NER 

In [ ]:
#take order smaller datafram = 100000
df=df.iloc[:10000,:]

In [ ]:
df.head(15)
df['poem_id']=df.index
df.head(15)

In [ ]:
df.rename(columns={'العصر': 'Generation','الشاعر':'poet_name','الديوان':'divan','القافية':'rhyme','البحر':'poem_type','الشطر الايسر':'left_side','الشطر الايمن':'right_side','البيت':'verse'}, inplace=True)
df = df.reindex(columns=['poem_id','Generation', 'poet_name', 'divan', 'rhyme', 'poem_type', 'left_side','right_side', 'verse'])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.apply(pd.Series.nunique)

In [ ]:
df.shape

In [ ]:
#df[df['poet_name']=='عمرو بنِ قُمَيئَة']
df.poet_name.value_counts().sort_values(ascending=False)

In [ ]:
df.rhyme.value_counts().sort_values(ascending=False)

أبيات القصيدة كلّها مَبنيّةً على تفعيلةٍ واحدةٍ تتبع لبحرٍ مُعيّنٍ من بحور الشّعر العربيّ.
وبحور الشّعر ستّة عشر بحراً وهي:
المُتدارَك، والمُتقارِب، والمُجتثّ، والمُقتَضَب، والمُضارِع، والخفيف، والمُنسرِح، والسّريع، والرَّمَل، والرَّجَز، والهَزَج، والكامل، والوافر، والبَسيط، والمَديد، والطّويل.

In [ ]:
df.poem_type.value_counts().sort_values(ascending=False)

In [ ]:
#df.groupby('rhyme')['poet_name'].value_counts().sort_values(ascending=False)
poet_rhyme =df.groupby('poet_name')['poem_type']
poet_rhyme.value_counts().sort_values(ascending=False)

In [ ]:
df.verse

In [ ]:
poet_rhyme = df.groupby(['poet_name','rhyme']).size().reset_index(name ='Count')
print(poet_rhyme)

In [ ]:
df.verse

In [ ]:
df.poet_name.value_counts().sort_values(ascending=False)

In [ ]:
df.sample(5)

# Classify each Poet name with his verses by using Logistic Regression

In [ ]:
x = df.verse
y = df.poet_name

In [ ]:
x_train_val,x_test,y_train_val,y_test = train_test_split(x,y, test_size=0.2, random_state= 30)
x_train,x_val,y_train,y_val = train_test_split(x_train_val,y_train_val, test_size=0.2, random_state= 30)

In [ ]:
# remove a special character _
x_train= x_train.apply(lambda x: re.sub(r'[_]+', '', x))
x_val= x_val.apply(lambda x: re.sub(r'[_]+', '', x))

# remove (non-arabic-alphabet)
x_train= x_train.str.replace('[^ء-ي ]', '')
x_val= x_val.str.replace('[^ء-ي ]', '')

# replcae (أ,آ,إ) by (ا)
x_train= x_train.apply(lambda x: re.sub('[أإآ]', 'ا', x))
x_val= x_val.apply(lambda x: re.sub('[أإآ]', 'ا', x))

# replace (ة) by (ه)
x_train = x_train.apply(lambda x: re.sub('[ة]', 'ه', x))
x_val = x_val.apply(lambda x: re.sub('[ة]', 'ه', x))

# remove multi spaces
x_train= x_train.apply(lambda x: re.sub(' +', ' ', x).strip())
x_val= x_val.apply(lambda x: re.sub(' +', ' ', x).strip())

In [ ]:
# remove punctuations
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
x_train = x_train.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
x_test = x_test.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))
x_val = x_val.apply(lambda x: x.translate(str.maketrans('', '', punctuations_list)))

In [ ]:
def deNoise(text):
    noise = re.compile(""" ّ      | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    return text

In [ ]:
#Text preprocessing for y_value
y_train = y_train.apply(lambda x: deNoise(x))
y_test = y_test.apply(lambda x: deNoise(x))
y_val= y_val.apply(lambda x: deNoise(x))

#For train Set
y_train= y_train.apply(lambda x: re.sub(r'[_]+', '', x))
y_train= y_train.str.replace('[^ء-ي ]', '')
y_train= y_train.apply(lambda x: re.sub('[أإآ]', 'ا', x))
y_train = y_train.apply(lambda x: re.sub('[ة]', 'ه', x))
y_train= y_train.apply(lambda x: re.sub(' +', ' ', x).strip())
#For validation set
y_val= y_val.apply(lambda x: re.sub(r'[_]+', '', x))
y_val= y_val.str.replace('[^ء-ي ]', '')
y_val= y_val.apply(lambda x: re.sub('[أإآ]', 'ا', x))
y_val = y_val.apply(lambda x: re.sub('[ة]', 'ه', x))
y_val= y_val.apply(lambda x: re.sub(' +', ' ', x).strip())

In [ ]:
stop_words = stopwords.words('arabic')
stop_words.extend(['هـ', 'د', 'م', 'الى', 'ان', 'اذ', 'لهذه', 'قال', 'وقال', 'اكد', 'عدد', 'بعدد', 'وعدد'
                  , 'والتي', 'بن', 'بنت', 'وقد', 'ا', 'عبر', 'خلال', 'او', 'الا', 'وان', 'اي', 'بان', 'كان'
                  , 'كانت' ,'تم','الف','مليون', 'وفي', 'وقد','اكثر','اقل', 'انه','وانه', 'قالت', 'وقالت', 'وتم'])

In [ ]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams
#from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(stop_words='arabic')

x_train_cv1 = cv1.fit_transform(x_train)
x_test_cv1  = cv1.transform(x_val)

pd.DataFrame(x_train_cv1.toarray(), columns=cv1.get_feature_names()).head()

# NLP -Unsupervised ML

In [ ]:
text_train =x_train['verse'].values

In [ ]:
set(stopwords.words('arabic'))

In [ ]:
text

# Tokenization

In [ ]:
#Sentence Tokenization
data = list(df['verse'].tolist())
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:3])

# Stop words removal & Bigrams

In [ ]:
stop_words = stopwords.words('arabic')
stop_words.extend(['هـ', 'د', 'م', 'الى', 'ان', 'اذ', 'لهذه', 'قال', 'وقال', 'اكد', 'عدد', 'بعدد', 'وعدد'
                  , 'والتي', 'بن', 'بنت', 'وقد', 'ا', 'عبر', 'خلال', 'او', 'الا', 'وان', 'اي', 'بان', 'كان'
                  , 'كانت' ,'تم','الف','مليون', 'وفي', 'وقد','اكثر','اقل', 'انه','وانه', 'قالت', 'وقالت', 'وتم'])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
remove_stpwrd=remove_stopwords(text)
remove_stpwrd

In [ ]:
#stemming as experiment
def apply_stemming_and_Stopwords(texts):
    st = ISRIStemmer()
    return [[st.stem(word) for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [ ]:
stemmstop_word=apply_stemming_and_Stopwords(text)
stemmstop_word

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
 
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
data_words_nostops = apply_stemming_and_Stopwords(data_words)
data_words_nostops

In [ ]:
 def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(remove_stpwrd)
data_words_bigrams

In [ ]:
data_words_trigrams = make_trigrams(data_words_bigrams)
data_words_trigrams

In [ ]:
unigrams  = (df['verse'].str.replace(r'[^a-z\s]', '').str.split(expand=True).stac

# generate bigrams by concatenating unigram columns
bigrams = unigrams + ' ' + unigrams.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
trigrams = bigrams + ' ' + unigrams.shift(-2)

# concatenate all series vertically, and remove NaNs
txt_ngram=pd.concat([unigrams, bigrams, trigrams]).dropna().reset_index(drop=True)
txt_ngram

In [ ]:
allowed_chars = ['ض', 'ص', 'ث', 'ق', 'ف', 'غ', 'ع', 'ه', 'خ', 'ح', 'ج', 'د', 'ش', 'س', 'ي', 'ب', 'ل', 'ا', 'ت', 'ن', 'م',
                 'ك', 'ط', 'ئ', 'ء', 'ؤ', 'ر' , 'ى', 'ة', 'و', 'ز', 'ظ', 'إ' ,'أ', 'آ', ' ', '\n', 'ذ', 'ّ']

In [ ]:
def delete_puncs(poem):
    for l in poem:
        if l not in allowed_chars:
            poem = poem.replace(l, "")
            return poem

def extract_poems(poet_name):
    with open('all_poems.csv'.format(poet_name), 'w', encoding="utf-8") as f:
        for i in range(len(df)):
            if df['poet_name'][i].format(poet_name) == poet_name:
                poem = delete_puncs(df['verse'][i])
                f.write("{}\n\n".format(poem)).verse

In [ ]:
df['rhyme'][257]

In [ ]:
all_poets = set()
for poet_name in df['verse']:
    all_poets.add(poet_name)

In [ ]:
after_filter = araby.strip_diacritics(text)

print(after_filter)

In [ ]:
#extract all words on the same rythmes

def rhymes_with(word):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله']:
        word = word.replace('ّ', '')
    ending = word[-2:]
    rhymes = []
    for w in voc_list:
        if len(w) < max_word_length and w.endswith(ending):
            rhymes.append(w)
    return rhymes

In [ ]:
def rhymes_with_last_n_chars(word, n):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله', 'فالله']:
        word = word.replace('ّ', '')
        ending = word[-n:]
        rhymes = []
        for w in voc_list:
            if len(w) < max_word_length and w.endswith(ending):
                rhymes.append(w)
                return rhymes

In [ ]:
extract_poems= df['poet_name'].apply(lambda text,x: rhymes_with_last_n_chars(text,x))

In [ ]:
import pickle

with open('vocabs.pkl', 'rb') as pickle_load:
    voc_list = pickle.load(pickle_load)

In [ ]:
import markovify
from tqdm import tqdm

def markov(text_file):
    with open(text_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text_model = markovify.NewlineText(text)
    return text_model

In [ ]:
def generate_poem_single_rhyme(poet_name, rhyme, iterations=3000, use_tqdm=False):
    n_of_rhyme_letters = len(rhyme)
    input_ff =  df.loc[df['poet_name'] == poet_name]
    input_file = '../input/poem-txt/.txt'
    text_model = markov(input_file)
    rhymes_list = rhymes_with_last_n_chars(rhyme, n_of_rhyme_letters)
    bayts = set()
    used_rhymes = set()
    
    poem = ""
    
    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)
        
    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]
        if (last_word in rhymes_list) and (last_word not in used_rhymes) and (bayt not in bayts):
            bayts.add(bayt)
            used_rhymes.add(last_word)
            poem += "{}\n".format(bayt)
            if not use_tqdm:
                print(bayt)
    return poem


In [ ]:
def generate_poem_2_rhymes(poet_name, rhyme_1, rhyme_2, iterations=3000, use_tqdm=False):
    n_of_rhyme_1_letters = len(rhyme_1)
    n_of_rhyme_2_letters = len(rhyme_2)
    
    input_ff =  df.loc[df['poet_name'] == poet_name]
    input_file = '../input/poem-txt/.txt'
    text_model = markov(input_file)
    
    rhymes_1_list = rhymes_with_last_n_chars(rhyme_1, n_of_rhyme_1_letters)
    rhymes_2_list = rhymes_with_last_n_chars(rhyme_2, n_of_rhyme_2_letters)
    
    bayts_1 = set()
    bayts_2 = set()
    
    used_rhymes_1 = set()
    used_rhymes_2 = set()
    
    poem = ""
    
    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)
        
    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]
        
        if (last_word in rhymes_1_list) and (last_word not in used_rhymes_1) and (bayt not in bayts_1):
            bayts_1.add(bayt)
            used_rhymes_1.add(last_word)

        if (last_word in rhymes_2_list) and (last_word not in used_rhymes_2) and (bayt not in bayts_2):
            bayts_2.add(bayt)
            used_rhymes_2.add(last_word)                
    
    len_of_poem = min(len(bayts_1), len(bayts_2))
    for i in range(len_of_poem):
        poem += "{}\n{}\n".format(list(bayts_1)[i], list(bayts_2)[i])
        
    return poem

In [ ]:
x = generate_poem_single_rhyme('ب','الأَخطَل', iterations=3000, use_tqdm=False)
x

# TF-IDF Vectorizer

In [ ]:
# the document-term matrix 
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
tfidf = TfidfVectorizer(stop_words=arb_stopwords)
doc_words = tfidf.fit_transform(df.verse)
tfidf_feature_name=pd.DataFrame(doc_words.toarray(),columns=tfidf.get_feature_names())
tfidf_feature_name

In [ ]:
?ArabicWordCloud

In [ ]:
from wordcloud import WordCloud
font_file = './NotoNaskhArabic-Regular.ttf'
wordcloud = WordCloud(width = 800, height = 800,min_font_size = 10,font_path=font_file).generate(doc_words)
wordcloud.to_file("arabic_example13.png")
#<wordcloud.wordcloud.WordCloud at 0x1ca1b4a9550>
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
?ArabicWordCloud

In [ ]:
# print WordCloud visualization

awc = ArabicWordCloud(background_color="black")

mpl.rcParams['figure.figsize']=(20,12.0)  
mpl.rcParams['font.size']=12            
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 
mpl.rcParams['figure.file']='NotoSansArabic-ExtraBold.ttf'

# wordcloud = awc.generate(str(tfidf.get_feature_names())).from_text(
#     str(tfidf.get_feature_names()))`

wordcloud = awc.generate(str(tfidf.get_feature_names())).from_text(str(tfidf.get_feature_names()))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud.recolor(colormap='Dark2'),interpolation='bilinear')
#plt.title("Word Cloud of all the words")
plt.axis('off')
plt.show();

# K-mean Model

In [ ]:
?TfidfVectorizer

In [ ]:
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))

In [ ]:
documents = df.verse.copy()

vectorizer = TfidfVectorizer(stop_words=arb_stopwords, max_df=8, min_df=2)
X = vectorizer.fit_transform(documents)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
true_k = 8
# model = KMeans(n_clusters=true_k, init='k-means++',random_state=60, max_iter=100, n_init=1)
model = KMeans(n_clusters=true_k, init='k-means++',random_state=60, max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),